In [1]:
import cv2
import numpy as np

In [37]:
img_path = 'D:/College/Sem VI/CV/lane4.jpg'
image = cv2.imread(img_path)
# cv2.imshow("Image",image)
# cv2.waitKey(0)

hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
lower = np.array([0,190,0]) # keep everything in this range, rest blackened
upper = np.array([255,255,255])
yellowlower = np.array([50,75,65]) # keep everything in this range, rest blackened
yellowupper = np.array([125,255,168])

# https://www.tydac.ch/color/
# helps calculate ranges for colors

# remove whatever isnt white coz lanes are white and we dont care about the rest
# hue(shade), lightness and saturation
whitemask = cv2.inRange(hls, lower, upper)
yellowmask = cv2.inRange(hls, yellowlower, yellowupper)
mask = cv2.bitwise_or(yellowmask, whitemask)  # yellow coz yellow lanes also i guess
masked = cv2.bitwise_and(image, image, mask = mask) 

cv2.imshow("Image",masked)
cv2.waitKey(0)

-1

In [38]:
def roi(img):
    x = int(img.shape[1])
    y = int(img.shape[0])
    shape = np.array([[int(0), int(y)], [int(x), int(y)], [int(0.55*x), int(0.55*y)], [int(0.50*x), int(0.55*y)]])
#     cordinates of polygon in the middle in which lane would be

    #define a numpy array with the dimensions of img, but comprised of zeros
    mask = np.zeros_like(img)

    #Uses 3 channels or 1 channel for color depending on input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #creates a polygon with the mask color
    cv2.fillPoly(mask, np.int32([shape]), ignore_mask_color)

    #returns the image only where the mask pixels are not zero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

roi_img = roi(masked)

cv2.imshow("Image",roi_img)
cv2.waitKey(0)

-1

In [39]:
# def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
#     """
#     `img` should be the output of a Canny transform.
#     """
#     lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
#     line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
#     draw_lines(line_img, lines)
#     return line_img

# def linedetect(img):
#     return hough_lines(img, 1, np.pi/180, 10, 20, 100)


edges = cv2.Canny(cv2.cvtColor(roi_img, cv2.COLOR_RGB2GRAY), 50, 120)
cv2.imshow("Canny",edges)
cv2.waitKey(0)


lines = cv2.HoughLinesP(edges,1,np.pi/180,30, 100, 1)
# img2=image.copy()
# for line in lines:
#   x1, y1, x2, y2 = line[0]
#   cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
# cv2.imshow("Image",image)
# cv2.waitKey(0)

In [40]:
rightColor=[0,255,0]
leftColor=[255,0,0]

# To draw lane lines we need to find the following things:
# - Declare slope and intercept for 2 lines - left lane line and right lane line
# - For each line in houghlines, determine whether its a left lane line of a right lane line
# - Append the slope and intercept to corresponsind arrays
# - Calculate the average slope and average intercept of left lane lines and right lane lines
# - Draw 2 lines on the image using these slops and averages
# 

rightSlope, leftSlope, rightIntercept, leftIntercept = [],[],[],[]
for line in lines:
    for x1,y1,x2,y2 in line:
        slope = (y1-y2)/(x1-x2)
        # only lines with slope > 0.3 can be considered as right lane lines
        if slope > 0.1:
            # if x coordinate of a line is grater than 500,
            # then its likely to a right lane line
            # positive slope coz y is postive downwards so not same logic as normal cartesian
            if x1 > 200 :
                # y = mx + c
                # c = y-mx
                yintercept = y2 - (slope*x2)
                rightSlope.append(slope)
                rightIntercept.append(yintercept)
            else: None
        # only lines with slope < -0.3 can be considered as left lane lines
        elif slope < -0.3:
            # if x coordinate of a line is less than 600,
            # then its likely to a left lane line
            if x1 < 600:
                yintercept = y2 - (slope*x2)
                leftSlope.append(slope)
                leftIntercept.append(yintercept)

leftavgSlope = np.mean(leftSlope[-30:])
leftavgIntercept = np.mean(leftIntercept[-30:])

rightavgSlope = np.mean(rightSlope[-30:])
rightavgIntercept = np.mean(rightIntercept[-30:])

# y = mx + c
# x = (y-c)/m
left_line_x1 = int((0.65*image.shape[0] - leftavgIntercept)/leftavgSlope)
left_line_x2 = int((image.shape[0] - leftavgIntercept)/leftavgSlope)

right_line_x1 = int((0.65*image.shape[0] - rightavgIntercept)/rightavgSlope)
right_line_x2 = int((image.shape[0] - rightavgIntercept)/rightavgSlope)

pts = np.array([[left_line_x1, int(0.65*image.shape[0])],[left_line_x2, int(image.shape[0])],[right_line_x2, int(image.shape[0])],[right_line_x1, int(0.65*image.shape[0])]], np.int32)
print(pts.shape)
pts = pts.reshape((-1,1,2))
print(pts.shape)
cv2.fillPoly(image,[pts],(0,0,255))

# cv2.imshow("fillpoly",image)

cv2.line(image, (left_line_x1, int(0.65*image.shape[0])), (left_line_x2, int(image.shape[0])), leftColor, 10)
# cv2.imshow("Left lane",image)
cv2.line(image, (right_line_x1, int(0.65*image.shape[0])), (right_line_x2, int(image.shape[0])), rightColor, 10)
# cv2.imshow("Right lane",image)

cv2.imshow("Image",image)
cv2.waitKey(0)

(4, 2)
(4, 1, 2)


-1

In [54]:
# Cartoonize
import cv2

img_path = 'D:/Jatayu Unmanned Systems/dataset2/New/Trump/Trump_7.jpg'
image = cv2.imread(img_path)
image = cv2.resize(image, None, fx=1.0/2, fy=1.0/2,interpolation=cv2.INTER_AREA)

img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Remove noise from the image using median filter 
img_gray = cv2.medianBlur(img_gray, 7)

# Blur the image using gaussian blur
img_gray = cv2.GaussianBlur(img_gray, (5, 5), 0)
cv2.imshow("Gray Blurred", img_gray)
cv2.waitKey(0)


# Apply Laplacian transform to create a mask for edges
edges = cv2.Laplacian(img_gray, cv2.CV_8U, ksize=5)
ret, mask = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY_INV)
cv2.imshow("Edges", edges)
cv2.waitKey(0)


# Resize the image to 1/4th of its dimensions
ds_factor = 4
num_repetitions = 15

img_small = cv2.resize(image, None, fx=1.0/ds_factor, fy=1.0/ds_factor,interpolation=cv2.INTER_AREA)

# Apply bilateral filter the image multiple times to acheice the cartoon effect
for i in range(num_repetitions):
    img_small = cv2.bilateralFilter(img_small, 5, 5, 7)

# Resize the image to original shape
img_output = cv2.resize(img_small, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
cv2.imshow("Color Blurred", img_output)
cv2.waitKey(0)

# Create a blank destination image 
dst = np.zeros(img_gray.shape)

# Apply bitwise and of the mask created earlier on the image
dst = cv2.bitwise_and(img_output, img_output, mask=mask)
cv2.imshow("Cartoon", dst)
cv2.waitKey(0)

(834, 1111, 3)
(208, 278, 3)
(834, 1111, 3)


-1